<a href="https://colab.research.google.com/github/BenJavaCode/3.sem-Benjamin/blob/master/bacteria_id.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch
import numpy as np

In [8]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [43]:
def remove_header(directory):
    data = []
    with open(directory, 'r',  encoding="utf8", errors='ignore') as f:
        data = f.readlines()
    for i,j in enumerate(data):
        if not j.startswith('#'):
            data = data[i:]
            for i,j in enumerate(data):
                data[i] = float(j[j.find('\t'):-1]) # crop to 1615
            data = data[0:-1]
            return(data)

In [44]:
from os import walk
def create_file_list(path):
    directories = []
    files = []
    for (dirpath, dirnames, filenames) in walk(path):
        directories.extend(dirnames)
        break
    for i in directories:
        for (dirpath, dirnames, filenames) in walk(f'{path}/{i}'):
            files.extend([path +'/'+i +'/'+ x for x in filenames])
            break
    
    return(files)


In [45]:
data_list = []
path_list = create_file_list('/content/gdrive/MyDrive/data_students')
labels = []
for i in path_list:
    data_list.append(remove_header(i))

#bulshit code
for i in path_list:
    if 'PDMS' in i:
        labels.append(0.0)
    if 'PMMA' in i:
        labels.append(1.0)
    if 'polystyrene' in i:
        labels.append(2.0)


In [46]:
len(data_list),len(labels)

(42, 42)

In [47]:
len(data_list[0])

1615

In [48]:
def normalize_by_each_dataset(dataset):
    for i in dataset:
        min_, max_ = min(i), max(i)
        for j, val in enumerate(i):
            i[j] = (val-min_)/(max_-min_)

'''The dataset of each compound
was normalized to zero mean and unit variance at each Raman
shift'''

'The dataset of each compound\nwas normalized to zero mean and unit variance at each Raman\nshift'

In [49]:
normalize_by_each_dataset(data_list)
data_list

[[0.9762219732993455,
  1.0,
  0.9707502584517113,
  0.9859218314383337,
  0.9496330056092117,
  0.9427487811825569,
  0.9269483833335133,
  0.8698567170931023,
  0.855778548531436,
  0.8251162974041265,
  0.7792871660130487,
  0.7567620963143825,
  0.7264142576182837,
  0.6974742357783514,
  0.655084870239174,
  0.6142581814103416,
  0.5757778540084535,
  0.5613852730155766,
  0.5225952259053319,
  0.4935003442112214,
  0.4770758142226107,
  0.4722235387916896,
  0.431866122248246,
  0.4115325541223459,
  0.3805605832866799,
  0.37539858814740223,
  0.3597563350585347,
  0.33551044388934753,
  0.3425495281701807,
  0.3220578152841045,
  0.3048514776680359,
  0.2821696710260498,
  0.2773206805011265,
  0.26465032879562683,
  0.25745497684375906,
  0.26027061055609235,
  0.24994662027753703,
  0.23070645657659303,
  0.22601373372270422,
  0.22523145682296095,
  0.20880692683435018,
  0.20114230259709365,
  0.19879594117014923,
  0.18878448623366287,
  0.1908183123185383,
  0.18565631717

In [50]:
class SpectralDataset(Dataset):
    """
    Builds a dataset of spectral data. Use idxs to specify which samples to use
    for dataset - this allows for random splitting into training, validation,
    and test sets. Instead of passing in filenames for X and y, we can also
    pass in numpy arrays directly.
    
    """
    def __init__(self, X_fn, y_fn, idxs=None, transform=None):
        if type(X_fn) == str:
            self.X = np.load(X_fn)
        else:
            self.X = X_fn
        if type(y_fn) == str:
            self.y = np.load(y_fn)
        else:
            self.y = y_fn
        if idxs is None: idxs = np.arange(len(self.y))
        self.idxs = idxs
        self.transform = transform

    def __len__(self):
        return len(self.idxs)

    def __getitem__(self, idx):
        i = self.idxs[idx]
        x, y = self.X[i], self.y[i]
        x = np.expand_dims(x, axis=0)
        if self.transform:
            x = self.transform(x)
        return (x, y)


### TRANSFORMS ###


class GetInterval(object):
    """
    Gets an interval of each spectrum.
    """
    def __init__(self, min_idx, max_idx):
        self.min_idx = min_idx
        self.max_idx = max_idx

    def __call__(self, x):
        x = x[:,self.min_idx:self.max_idx]
        return x


class ToFloatTensor(object):
    """
    Converts numpy arrays to float Variables in Pytorch.
    """
    def __call__(self, x):
        x = torch.from_numpy(x).float()
        return x


### TRANSFORMS ###


def spectral_dataloader(X_fn, y_fn, idxs=None, batch_size = 2, shuffle=True,
    num_workers=2, min_idx=None, max_idx=None, sampler=None):#batch size altered from 10  to 4
    """
    Returns a DataLoader with spectral data.
    """
    transform_list = []
    if min_idx is not None and max_idx is not None:
        transform_list.append(GetInterval(min_idx, max_idx))
    transform_list.append(ToFloatTensor())
    transform = transforms.Compose(transform_list)
    dataset = SpectralDataset(X_fn, y_fn, idxs=idxs, transform=transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle,
        num_workers=num_workers, sampler=sampler)
    return dataloader


def spectral_dataloaders(X_fn, y_fn, n_train=None, p_train=0.8, p_val=0.1,
    n_test=None, batch_size=2, shuffle=True, num_workers=2, min_idx=None,
    max_idx=None):#batch size altered from 10  to 4
    """
    Returns train, val, and test DataLoaders by splitting the dataset randomly.
    Can also take X_fn and y_fn as numpy arrays.
    """
    if type(y_fn) == str:
        idxs = np.arange(len(np.load(y_fn)))
    else:
        idxs = np.arange(len(y_fn))
    np.random.shuffle(idxs)
    if n_train is None: n_train = int(p_train * len(idxs))
    n_val = int(p_val * n_train)
    val_idxs, train_idxs = idxs[:n_val], idxs[n_val:n_train]
    if n_test is None: test_idxs = idxs[n_train:]
    else: test_idxs = idxs[n_train:n_train+n_test]
    trainloader = spectral_dataloader(X_fn, y_fn, train_idxs,
        batch_size=batch_size, shuffle=shuffle, num_workers=num_workers,
        min_idx=min_idx, max_idx=max_idx)
    valloader = spectral_dataloader(X_fn, y_fn, val_idxs,
        batch_size=batch_size, shuffle=shuffle, num_workers=num_workers,
        min_idx=min_idx, max_idx=max_idx)
    testloader = spectral_dataloader(X_fn, y_fn, test_idxs,
        batch_size=batch_size, shuffle=shuffle, num_workers=num_workers,
        min_idx=min_idx, max_idx=max_idx)
    return (trainloader, valloader, testloader)

In [51]:
from time import time
t00 = time()
import numpy as np

#X_fn = '/content/gdrive/MyDrive/bacteria_id_data/data/unzipped_data/X_finetune.npy'
#y_fn = '/content/gdrive/MyDrive/bacteria_id_data/data/unzipped_data/y_finetune.npy'

#X = np.load(X_fn)
#y = np.load(y_fn)

X = np.array(data_list)
y = np.array(labels)
print(X.shape, y.shape)

(42, 1615) (42,)


In [52]:
import torchvision.models
import os
import torch

In [53]:
# CNN parameters
layers = 6
hidden_size = 14 #altered from 100 to 14
block_size = 2
hidden_sizes = [hidden_size] * layers
num_blocks = [block_size] * layers
input_dim = 1615 #altered from 1000 to 1616
in_channels = 64
n_classes = 3 #Aletered from original (30)
os.environ['CUDA_VISIBLE_DEVICES'] = '{}'.format(0)
cuda = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



In [54]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable


class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()

        # Layers
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=5,
            stride=stride, padding=2, dilation=1, bias=False) #KERNEL_SIZE ALTERED FROM 5 TO 4
        self.bn1 = nn.BatchNorm1d(num_features=out_channels)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=5,
            stride=1, padding=2, dilation=1, bias=False) #kERNEL SIZE ALTERED FROM 5 TO 4
        self.bn2 = nn.BatchNorm1d(num_features=out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, kernel_size=1,
                    stride=stride, bias=False),
                nn.BatchNorm1d(out_channels))#KERNEL_SIZE ALTERED FROM 1 TO 2

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, hidden_sizes, num_blocks, input_dim=1615, #altered from 1000 to 1616
        in_channels=64, n_classes=3):## n_classes aletered from original (30)
        super(ResNet, self).__init__()
        assert len(num_blocks) == len(hidden_sizes)
        self.input_dim = input_dim
        self.in_channels = in_channels
        self.n_classes = n_classes
        
        self.conv1 = nn.Conv1d(1, self.in_channels, kernel_size=5, stride=1,
            padding=2, bias=False) #kERNEL SIZE ALTERED FROM 5 TO 4
        self.bn1 = nn.BatchNorm1d(self.in_channels)
        
        # Flexible number of residual encoding layers
        layers = []
        strides = [1] + [2] * (len(hidden_sizes) - 1)
        for idx, hidden_size in enumerate(hidden_sizes):
            layers.append(self._make_layer(hidden_size, num_blocks[idx],
                stride=strides[idx]))
        self.encoder = nn.Sequential(*layers)

        self.z_dim = self._get_encoding_size()
        self.linear = nn.Linear(self.z_dim, self.n_classes)


    def encode(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.encoder(x)
        z = x.view(x.size(0), -1)
        return z

    def forward(self, x):
        z = self.encode(x)
        return self.linear(z)


    def _make_layer(self, out_channels, num_blocks, stride=1):
        strides = [stride] + [1] * (num_blocks - 1)
        blocks = []
        for stride in strides:
            blocks.append(ResidualBlock(self.in_channels, out_channels,
                stride=stride))
            self.in_channels = out_channels
        return nn.Sequential(*blocks)

    def _get_encoding_size(self):
        """
        Returns the dimension of the encoded input.
        """
        temp = Variable(torch.rand(1, 1, self.input_dim))
        z = self.encode(temp)
        z_dim = z.data.size(1)
        return z_dim


def add_activation(activation='relu'):
    """
    Adds specified activation layer, choices include:
    - 'relu'
    - 'elu' (alpha)
    - 'selu'
    - 'leaky relu' (negative_slope)
    - 'sigmoid'
    - 'tanh'
    - 'softplus' (beta, threshold)
    """
    if activation == 'relu':
        return nn.ReLU()
    elif activation == 'elu':
        return nn.ELU(alpha=1.0)
    elif activation == 'selu':
        return nn.SELU()
    elif activation == 'leaky relu':
        return nn.LeakyReLU(negative_slope=0.1)
    elif activation == 'sigmoid':
        return nn.Sigmoid()
    elif activation == 'tanh':
        return nn.Tanh()
    # SOFTPLUS DOESN'T WORK with automatic differentiation in pytorch
    elif activation == 'softplus':
        return nn.Softplus(beta=1, threshold=20)

In [67]:
# Load trained weights for demo
cnn = ResNet(hidden_sizes, num_blocks, input_dim=input_dim,
                in_channels=in_channels, n_classes=n_classes)
if cuda: cnn.cuda()
#Find out if it is possible to reshape weights, and if it offers any advatages.
'''
cnn.load_state_dict(torch.load(
    '/content/gdrive/MyDrive/bacteria_id_data/pretrained_model.ckpt', map_location=lambda storage, loc: storage))
'''

"\ncnn.load_state_dict(torch.load(\n    '/content/gdrive/MyDrive/bacteria_id_data/pretrained_model.ckpt', map_location=lambda storage, loc: storage))\n"

In [68]:
from torch.autograd import Variable
from torch import nn
import torch
import numpy as np


def run_epoch(epoch, model, dataloader, cuda, training=False, optimizer=None):
    if training:
        model.train()
    else:
        model.eval()
    total_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(dataloader):
        if cuda: inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = Variable(inputs), Variable(targets.long())
        outputs = model(inputs)
        loss = nn.CrossEntropyLoss()(outputs, targets)
        if training:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        if cuda:
            correct += predicted.eq(targets.data).cpu().sum().item()
        else:
            correct += predicted.eq(targets.data).sum().item()
    acc = 100 * correct / total
    avg_loss = total_loss / total
    return acc, avg_loss


def get_predictions(model, dataloader, cuda, get_probs=False):
    preds = []
    model.eval()
    for batch_idx, (inputs, targets) in enumerate(dataloader):
        if cuda: inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = Variable(inputs), Variable(targets.long())
        outputs = model(inputs)
        if get_probs:
            probs = torch.nn.functional.softmax(outputs, dim=1)
            if cuda: probs = probs.data.cpu().numpy()
            else: probs = probs.data.numpy()
            preds.append(probs)
        else:
            _, predicted = torch.max(outputs.data, 1)
            if cuda: predicted = predicted.cpu()
            preds += list(predicted.numpy().ravel())
    if get_probs:
        return np.vstack(preds)
    else:
        return np.array(preds)

In [69]:
p_val = 0.2 #altered from 0.1 to 0.2
n_val = int(42 * p_val) #altered from 3000 to 44
idx_tr = list(range(42)) #altered from 3000 to 44
np.random.shuffle(idx_tr)
idx_val = idx_tr[:n_val]
idx_tr = idx_tr[n_val:]

In [70]:
#tests for sanity
idx_tr = [0,1,2,3,4,5,6,7,8,9,10,11,14,15,16,17,18,19,20,21,22,23,24,25,28,29,30,31,32,33,34,35,36,37,38,39]
idx_val = [12,13,26,27,40,41]

In [71]:
idx_val, idx_tr

([12, 13, 26, 27, 40, 41],
 [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39])

In [72]:
from torch import optim

In [73]:
# Fine-tune CNN
epochs = 10 # Change this number to ~30 for full training
batch_size = 2
t0 = time()
# Set up Adam optimizer
optimizer = optim.Adam(cnn.parameters(), lr=1e-3, betas=(0.5, 0.999))
# Set up dataloaders
dl_tr = spectral_dataloader(X, y, idxs=idx_tr,
    batch_size=batch_size, shuffle=True)
dl_val = spectral_dataloader(X, y, idxs=idx_val,
    batch_size=batch_size, shuffle=False)
# Fine-tune CNN for first fold
best_val = 0
no_improvement = 0
max_no_improvement = 5
print('Starting fine-tuning!')
for epoch in range(epochs):
    print(' Epoch {}: {:0.2f}s'.format(epoch+1, time()-t0))
    # Train
    acc_tr, loss_tr = run_epoch(epoch, cnn, dl_tr, cuda,
        training=True, optimizer=optimizer)
    print('  Train acc: {:0.2f}'.format(acc_tr))
    # Val
    acc_val, loss_val = run_epoch(epoch, cnn, dl_val, cuda,
        training=False, optimizer=optimizer)
    print('  Val acc  : {:0.2f}'.format(acc_val))
    # Check performance for early stopping
    if acc_val > best_val or epoch == 0:
        best_val = acc_val
        no_improvement = 0
    else:
        no_improvement += 1
    if no_improvement >= max_no_improvement:
        print('Finished after {} epochs!'.format(epoch+1))
        break

print('\n This demo was completed in: {:0.2f}s'.format(time()-t00))

Starting fine-tuning!
 Epoch 1: 0.00s
  Train acc: 44.44
  Val acc  : 33.33
 Epoch 2: 0.69s
  Train acc: 100.00
  Val acc  : 33.33
 Epoch 3: 1.40s
  Train acc: 100.00
  Val acc  : 50.00
 Epoch 4: 2.11s
  Train acc: 100.00
  Val acc  : 50.00
 Epoch 5: 2.82s
  Train acc: 100.00
  Val acc  : 50.00
 Epoch 6: 3.52s
  Train acc: 100.00
  Val acc  : 50.00
 Epoch 7: 4.25s
  Train acc: 100.00
  Val acc  : 50.00
 Epoch 8: 4.99s
  Train acc: 100.00
  Val acc  : 50.00
Finished after 8 epochs!

 This demo was completed in: 608.28s
